In [15]:
# load results
import numpy as np

resnet50_predictions = np.load("resnet50_predictions.npz")["all_prediction"]
resnet50_predictions

array([[1.82925293e-03, 3.60626122e-03, 2.91937470e-01, ...,
        1.77926780e-03, 8.04474112e-04, 5.87729795e-04],
       [3.04410059e-04, 3.27440468e-03, 4.38404502e-03, ...,
        3.95579700e-04, 3.98502825e-03, 3.35717195e-04],
       [9.99932289e-01, 4.86021578e-01, 1.98667333e-03, ...,
        5.51156819e-01, 3.92024027e-04, 3.52333125e-04],
       ...,
       [1.38029223e-03, 1.07545369e-04, 3.70080161e-05, ...,
        3.20630819e-01, 7.77939131e-05, 2.33896862e-04],
       [2.71423021e-03, 2.76995380e-03, 2.30300629e-05, ...,
        5.58950414e-04, 6.77466113e-03, 1.04199418e-04],
       [3.13112442e-03, 6.86676567e-03, 4.85802814e-03, ...,
        2.36948044e-03, 1.50912325e-03, 9.24084987e-03]])

In [16]:
import pandas as pd


from dataset import PosterDataset

df = pd.read_csv(PosterDataset.csv_file)
true_values = df.iloc[:, -20:-1].values
genres = df.columns[-20:-1]
genres = list(map(lambda x: x[1:], genres))
single_genres = df.genre_count == 1

In [17]:
from sklearn.metrics import accuracy_score, f1_score

# calculate accuracy
single_prediction = (
    resnet50_predictions[single_genres]
    == resnet50_predictions.max(axis=1)[single_genres][:, None]
)
accuracy_score(single_prediction, true_values[single_genres]), f1_score(
    single_prediction, true_values[single_genres], average="samples"
)

(0.5385234998750089, 0.5385234998750089)

### 3 Single Label


In [18]:
three_columns = ["Action", "Romance", "Thriller"]
three_genres = np.isin(genres, three_columns)
three_genres_index_mask = (
    true_values[:, three_genres].sum(axis=1) >= 1
) & single_genres
three_genres_index_mask.sum()

15976

In [19]:
from sklearn.metrics import accuracy_score, f1_score

# calculate accuracy
three_single_prediction = resnet50_predictions[three_genres_index_mask,][
    :, three_genres
]
three_single_prediction_classes = (
    three_single_prediction == three_single_prediction.max(axis=1)[:, None]
)

accuracy_score(
    three_single_prediction_classes,
    true_values[three_genres_index_mask,][:, three_genres],
), f1_score(
    three_single_prediction_classes,
    true_values[three_genres_index_mask,][:, three_genres],
    average="samples",
)

(0.6861542313470206, 0.6861542313470206)

In [23]:
three_single_prediction_classes.sum(axis=0) / three_single_prediction.shape[0]

array([0.35127692, 0.32185779, 0.3268653 ])